In [1]:
from nnsight import LanguageModel
from typing import List, Callable
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import clear_output

clear_output()

In [2]:
# Load gpt2

model = LanguageModel("openai-community/gpt2", device_map="auto", dispatch=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [3]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): WrapperModule()
)


In [8]:
with model.trace() as tracer:

    with tracer.invoke("The Eiffel Tower is in the city of we are adding more and more tokens here"):

        # Ablate the last MLP for only this batch.
        for layer in model.transformer.h:
            print(layer.mlp.output.shape)
        model.transformer.h[-1].mlp.output[0][:] = 0

        # Get the output for only the intervened on batch.
        token_ids_intervention = model.lm_head.output.argmax(dim=-1).save()

    with tracer.invoke("The Eiffel Tower is in the city of"):

        # Get the output for only the original batch.
        token_ids_original = model.lm_head.output.argmax(dim=-1).save()

print("Original token IDs:", token_ids_original)
print("Intervention token IDs:", token_ids_intervention)

print("Original prediction:", model.tokenizer.decode(token_ids_original[0][-1]))
print("Intervention prediction:", model.tokenizer.decode(token_ids_intervention[0][-1]))

torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
torch.Size([1, 18, 768])
Original token IDs: tensor([[ 198,    1,    1,    1,    1,    1,    1,    1,  464,  412,  733,  417,
          417,  271,  287,  262, 1748,  262]], device='cuda:0')
Intervention token IDs: tensor([[  262,    12,   417,  8765,    11,   257,   262,  3504,   338,  3576,
          2528,   287,   257,   621,   517, 18028,   284,   284]],
       device='cuda:0')
Original prediction:  the
Intervention prediction:  to
